In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
import peakutils
from matplotlib import colors, cm
from scipy.signal import decimate
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.value_change_utils import open_experiment, CustomAlignedDataRewardBlocks, get_all_experimental_records, get_block_change_info, add_traces_and_peaks, plot_mean_trace_for_condition, one_session_get_block_changes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
mouse_id = 'SNL_photo22'
date = '20200917'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
behavioural_data, session_data = open_experiment(experiment_to_process)

In [50]:
all_reward_block_data = []
for reward_block in range(1,6):
    print(reward_block)
    try:
        params = {'state_type_of_interest': 5,
            'outcome': 2,
            'last_outcome': 0,  # NOT USED CURRENTLY
            'no_repeats' : 0,
            'last_response': 0,
            'align_to' : 'Time start',
            'instance':- 1,
            'plot_range': [-6, 6],
            'first_choice_correct': 0,
             'cue': 'None'}
        all_reward_block_data.append(CustomAlignedDataRewardBlocks(session_data, params, reward_block))
    except:
        pass

1
(51, 160000)
2
(34, 160000)
3
(23, 160000)
4
(53, 160000)
5
(20, 160000)


In [64]:
colours = cm.inferno(np.linspace(0, 1, 5))
fig, ax = plt.subplots(1,1)
for trial_type in range(1,6):
    try:
        one_side_data = all_reward_block_data[trial_type-1].contra_data
        mean_trace = decimate(one_side_data.mean_trace, 10)
        time_points = decimate(one_side_data.time_points, 10)
        traces = decimate(one_side_data.sorted_traces, 10)
        ax.plot(time_points, mean_trace, lw=1.5, color=colours[trial_type-1], label= trial_type)
        ax.axvline(0, color='k')
    except:
        pass
plt.legend()


In [5]:
block_types = pd.DataFrame({'block type': [1, 2, 3, 4, 5], 'left reward': [6, 4, 2, 2, 2], 'right reward': [2, 2, 2, 4, 6]})

In [41]:
mouse_id = 'SNL_photo21'
sessions = ['20200917', '20200918', '20200921']
for session_num, date in enumerate(sessions):
    block_change_info = one_session_get_block_changes(mouse_id, date, block_types)
    if session_num == 0:
        all_block_changes = block_change_info
    else:
         all_block_changes = pd.concat([all_block_changes,block_change_info], ignore_index=True)

(106, 160000)
189    20.0
Name: Trial num, dtype: float64
197    21.0
Name: Trial num, dtype: float64
415    40.0
Name: Trial num, dtype: float64
425    41.0
Name: Trial num, dtype: float64
620    60.0
Name: Trial num, dtype: float64
628    61.0
Name: Trial num, dtype: float64
635    62.0
Name: Trial num, dtype: float64
836    80.0
Name: Trial num, dtype: float64
844    81.0
Name: Trial num, dtype: float64
852    82.0
Name: Trial num, dtype: float64
860    83.0
Name: Trial num, dtype: float64
1027    100.0
Name: Trial num, dtype: float64
1453    140.0
Name: Trial num, dtype: float64
1461    141.0
Name: Trial num, dtype: float64
1652    160.0
Name: Trial num, dtype: float64
2051    200.0
Name: Trial num, dtype: float64
2066    201.0
Name: Trial num, dtype: float64
2236    220.0
Name: Trial num, dtype: float64
(186, 160000)
615    60.0
Name: Trial num, dtype: float64
623    61.0
Name: Trial num, dtype: float64
991    100.0
Name: Trial num, dtype: float64
999    101.0
Name: Trial num, dty

IndexError: index 0 is out of bounds for axis 0 with size 0

In [31]:
time_points = all_trials_traces.contra_data.time_points
plot_mean_trace_for_condition(all_block_changes, time_points, 'new reward amounts')
plot_mean_trace_for_condition(all_block_changes, time_points, 'change in reward amounts')
plot_mean_trace_for_condition(all_block_changes, time_points, 'new relative value')
plot_mean_trace_for_condition(all_block_changes, time_points, 'change in relative value')

In [30]:
reward_amounts = np.sort(block_change_info['new relative value'].unique())
colours = cm.inferno(np.linspace(0, 0.8, reward_amounts.shape[0]))
fig, ax = plt.subplots(1,1)
for reward_indx, reward_amount in enumerate(reward_amounts):
    time_points = decimate(all_trials_traces.contra_data.time_points, 10)
    rows = block_change_info[(block_change_info['new relative value']==reward_amount)]
    traces = rows['traces'].values
    flat_traces = np.zeros([traces.shape[0], traces[0].shape[1]])
    for idx, trace in enumerate(traces):
        flat_traces[idx, :] = trace
    mean_trace = decimate(np.mean(flat_traces, axis=0), 10)

                       
    ax.plot(time_points, mean_trace, lw=1.5, color=colours[reward_indx], label= reward_amount)
ax.axvline(0, color='k')
ax.set_xlim([-2,2])
plt.legend()

In [86]:
reward_amounts = np.sort(block_change_info['change in relative value'].unique())
colours = cm.inferno(np.linspace(0, 0.8, reward_amounts.shape[0]))
fig, ax = plt.subplots(1,1)
for reward_indx, reward_amount in enumerate(reward_amounts):
    time_points = decimate(all_trials_traces.contra_data.time_points, 10)
    rows = block_change_info[(block_change_info['change in relative value']==reward_amount)]
    traces = rows['traces'].values
    flat_traces = np.zeros([traces.shape[0], traces[0].shape[1]])
    for idx, trace in enumerate(traces):
        flat_traces[idx, :] = trace
    mean_trace = decimate(np.mean(flat_traces, axis=0), 10)

                       
    ax.plot(time_points, mean_trace, lw=1.5, color=colours[reward_indx], label= reward_amount)
ax.axvline(0, color='k')
ax.set_xlim([-2,2])
plt.legend()

In [28]:
sns.scatterplot(x='new reward amounts', y='peak size', data=block_change_info)

In [29]:
sns.scatterplot(x='change in reward amounts', y='peak size', data=block_change_info)

In [30]:
sns.scatterplot(x='new relative value', y='peak size', data=block_change_info)

In [31]:
sns.scatterplot(x='change in relative value', y='peak size', data=block_change_info)

In [65]:
colours = cm.inferno(np.linspace(0,1, 5))
fig2, ax = plt.subplots(1,3)
for trial_type in range(1,6):
    try:
        one_side_data = all_reward_block_data[trial_type-1].contra_data
        x_vals = np.ones([len(one_side_data.trial_peaks)])*trial_type
        ax[0].hist(one_side_data.trial_peaks, color=colours[trial_type-1], alpha=0.4)
        ax[1].scatter(trial_type, np.mean(one_side_data.trial_peaks), color=colours[trial_type-1])
        ax[1].set_ylabel('Mean peak')
        ax[2].scatter(trial_type, np.median(one_side_data.trial_peaks), color=colours[trial_type-1])
        ax[2].set_ylabel('Median peak')
        print(np.mean(one_side_data.trial_peaks), np.median(one_side_data.trial_peaks))
    except:
        pass

0.8719974503007026 0.8838530247845447
0.734418515406605 0.5818760037034908
1.3165566302983154 1.1614116229862628
0.7800667694836998 0.663554277871089
0.8304953025800229 0.5631001717818181


In [217]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

0.15315428214827714

In [ ]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

In [209]:
np.asarray(one_side_data.trial_peaks)/4

array([0.14386391, 0.36703822, 0.03294286, 0.67309225, 0.1799292 ,
       0.26867563, 0.06190406])

In [199]:
np.median([0.13636681, 0.46637622, 0.51340394])

0.46637622

In [207]:
np.median(np.concatenate([np.asarray(one_side_data.trial_peaks),np.asarray([0.13636681, 0.46637622, 0.51340394])]))

0.5444297804051131